In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from Bio import SeqIO
from Bio.Data import IUPACData 
import csv
import numpy as np
import tensorflow as tf

#data_path = 'features_CENH3_DMR6_LUCA-CHLRE00002_orthologues.csv'
data_path = 'features_oma-seqs-viridiplantae_test-4-5.csv'

Using TensorFlow backend.


In [2]:
def protein2integer(in_seq):
    
    ## define universe of possible input values
    all_protein_letters = list(IUPACData.extended_protein_letters)
    #print(all_protein_letters)
    ## define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(all_protein_letters))
    int_to_char = dict((i, c) for i, c in enumerate(all_protein_letters))
    ## integer encode input data
    integer_encoded = [char_to_int[char] for char in in_seq.upper()]
    
    #return(integer_encoded,len(all_protein_letters))
    return(integer_encoded)
    

In [3]:
def make_dataset(in_file):
    with open(in_file, 'r') as f:
        reader = csv.reader(f, delimiter="\t")
        # get all the rows as a list
        d_set = list(reader)
        # transform data into numpy array
        d_set = np.array(d_set).astype(str)
        
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    
    G = d_set[:, 0]
    X = integer_encoded_proteins
    Y = d_set[:, 2].astype(int)
                         
    return(d_set,G,X,Y)

In [14]:
def make_train_test_set(G,X,Y):
    indices = np.random.permutation(X.shape[0])
    train_size = int(indices.size*0.90)
    train_idx, test_idx = indices[:train_size], indices[train_size:]
    #print(len(train_idx),len(test_idx))
    
    X_train, X_test = X[train_idx,], X[test_idx,]
    #print(X_train.shape,X_test.shape)
    
    y_train, y_test = Y[train_idx,], Y[test_idx,]
    
    #print(X[train_idx[0],])
    #print(Y[train_idx[0],])

    #print(X_train[0,])
    #print(y_train[0,])
    
    return(X_train,y_train,X_test,y_test)

In [18]:
def model1(X_train_new, y_train,X_test_new, y_test):
    # https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
    # create the model
    embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(X_train_new, y_train, epochs=10, batch_size=100) #epochs=3, batch_size=64)
    # Final evaluation of the model
    scores = model.evaluate(X_test_new, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    return()


In [6]:
def model2(X_train_new, y_train,X_test_new, y_test):
    # Initializing the Sequential model from KERAS.
    model = Sequential()

    max_review_length = len(max(X, key=len))

    # Creating a 16 neuron hidden layer with Linear Rectified activation function.
    #model.add(Dense(16, input_dim=1, init='uniform', activation='relu'))
    model.add(Dense(16, input_dim=max_review_length, kernel_initializer='uniform', activation='relu'))

    # Creating a 8 neuron hidden layer.
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))

    # Adding a output layer.
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    # Compiling the model
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

    print(model.summary())

    # Fitting the model
    #model.fit(X, Y, nb_epoch=150, batch_size=10)
    #scores = model.evaluate(X, Y)
    model.fit(X_train_new, y_train, epochs=1000, batch_size=100)
    scores = model.evaluate(X_test_new, y_test, verbose=0)

    print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    
    return()

In [21]:
def one_hot_matrix(labels,C):
    
    C = tf.constant(C,name="C")
    one_hot_matrix = tf.one_hot(labels,C,axis=1)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [ ]:
def model3(X_train_new, y_train,X_test_new, y_test):
    # Number of nodes in each NN hidden layer
    n_nodes_hl1 = 1500
    n_nodes_hl2 = 1500
    n_nodes_hl3 = 1500

    # Number of orthology clusters
    #n_classes = len(np.unique(np.concatenate((y_train,y_test),axis=0)))     #2 or 3 or ...
    
    #y_all = np.concatenate((y_train,y_test),axis=0)
    #y_min = np.amin(y_all)
    #n_classes = np.amax(y_all-y_min)+1
    n_classes = int(np.amax(np.concatenate((y_train,y_test),axis=0))+1)
    
    train_y = one_hot_matrix(y_train,n_classes)
    test_y = one_hot_matrix(y_test,n_classes)

    # Batch size and Epoch size for training the NN
    batch_size = 100   #100
    hm_epochs = 500    #1000

    # Initializing X and Y
    x = tf.placeholder('float')
    y = tf.placeholder('float')

    # Initializing NN layers
    hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(X_train_new[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'f_fum':n_nodes_hl3,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


    
    
    l1 = tf.add(tf.matmul(x,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weight']), hidden_3_layer['bias'])
    l3 = tf.nn.relu(l3)

    prediction = tf.matmul(l3,output_layer['weight']) + output_layer['bias']

        
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #try:
        #    epoch = int(open(tf_log,'r').read().split('\n')[-2])+1
        #    print('STARTING:',epoch)
        #except:
        #    epoch = 1
        epoch = 1

        while epoch <= hm_epochs:
            epoch_loss = 1
            
            i=0
            while i < len(X_train_new):
                start = i
                end = i+batch_size
                batch_x = np.array(X_train_new[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
                epoch_loss += c
                i+=batch_size
                
            
            print('Epoch ',epoch,' out of ',hm_epochs,'- loss:',epoch_loss)
 
            
            #with open(tf_log,'a') as f:
            #    f.write(str(epoch)+'\n') 
            epoch +=1
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        #print("\nModel saved in path: %s " % my_model_save_path)
        print('\nAccuracy:',accuracy.eval({x:X_test_new, y:test_y}))
    return()

In [9]:
def test_1(d_set):
    print(d_set[:,0])
    print(d_set[0:2,1])
    print(d_set[:,2].astype(int))
    print(d_set.shape)
    print(d_set[:,1].shape)
    return()

In [10]:
def test_2(d_set):
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    print(len(integer_encoded_proteins))
    print(integer_encoded_proteins[0])
    #np.array(integer_encoded_proteins).shape
    print(integer_encoded_proteins.shape)
    #protein2integer(dataset[:,1])
    return()

In [11]:
def test_3(G,X,Y):
    print(G.shape)
    print(X.shape)
    print(Y.shape)

    print(G[0:3,])
    print(X[0:3,])
    print(Y[0:3,])
    
    print('Maximum review length: {}'.format(len(max(X, key=len))))
    print('Minimum review length: {}'.format(len(min(X, key=len))))
    
    return()

In [12]:
def test_4(X_train_new,X_train):
    print(X_train_new.shape)
    print(X_train_new[0,:])
    print(X_train.shape)
    print(X_train[0,])
    return()


In [15]:
dataset, G, X, Y = make_dataset(data_path)
X_train,y_train,X_test,y_test = make_train_test_set(G,X,Y)

#print("============ Test 1 =======================")
#test_1(dataset)
#print("============ Test 2 =======================")
#test_2(dataset)
#print("============ Test 3 =======================")
#test_3(G,X,Y)



In [16]:
top_words = len(list(IUPACData.extended_protein_letters)) # = 26
max_review_length = len(max(X, key=len)) # = 500
# truncate and pad input sequences
X_train_new = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test_new = sequence.pad_sequences(X_test, maxlen=max_review_length)
  
#print("============ Test 4 =======================")
#test_4(X_train_new,X_train)


In [19]:
model1(X_train_new, y_train,X_test_new, y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2684, 32)          832       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2684, 32)          3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1342, 32)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 57,237
Trainable params: 57,237
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
3796/3796 [==============================] - 51s 13ms/step - loss: -25594.9368 - acc: 0.0000e+00
Epoch 2/10
3796/3796 [===

()

In [17]:
model2(X_train_new, y_train,X_test_new, y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                42960     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 43,105
Trainable params: 43,105
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
3796/3796 [==============================] - 0s 84us/step - loss: -16675.6478 - acc: 0.0000e+00
Epoch 2/1000
3796/3796 [==============================] - 0s 25us/step - loss: -67045.9872 - acc: 0.0000e+00
Epoch 3/1000
3796/3796 [==============================] - 0s 26us/step - loss: -70546.3418 - acc: 0.0000e+00
Epoch 4/1000
3796/3796 [==============================] - 0s 24us/step

3796/3796 [==============================] - 0s 33us/step - loss: -70699.0912 - acc: 0.0000e+00
Epoch 69/1000
3796/3796 [==============================] - 0s 26us/step - loss: -70699.0902 - acc: 0.0000e+00
Epoch 70/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0916 - acc: 0.0000e+00
Epoch 71/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0912 - acc: 0.0000e+00
Epoch 72/1000
3796/3796 [==============================] - 0s 26us/step - loss: -70699.0908 - acc: 0.0000e+00
Epoch 73/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0906 - acc: 0.0000e+00
Epoch 74/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0908 - acc: 0.0000e+00
Epoch 75/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0926 - acc: 0.0000e+00
Epoch 76/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0914 - acc: 0.0000e+00
Epoch 77/1000
3796/3796 

3796/3796 [==============================] - 0s 24us/step - loss: -70699.0912 - acc: 0.0000e+00
Epoch 143/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0906 - acc: 0.0000e+00
Epoch 144/1000
3796/3796 [==============================] - 0s 26us/step - loss: -70699.0917 - acc: 0.0000e+00
Epoch 145/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0915 - acc: 0.0000e+00
Epoch 146/1000
3796/3796 [==============================] - 0s 23us/step - loss: -70699.0899 - acc: 0.0000e+00
Epoch 147/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0917 - acc: 0.0000e+00
Epoch 148/1000
3796/3796 [==============================] - 0s 27us/step - loss: -70699.0910 - acc: 0.0000e+00
Epoch 149/1000
3796/3796 [==============================] - 0s 33us/step - loss: -70699.0916 - acc: 0.0000e+00
Epoch 150/1000
3796/3796 [==============================] - 0s 33us/step - loss: -70699.0909 - acc: 0.0000e+00
Epoch 151/1000
3

3796/3796 [==============================] - 0s 26us/step - loss: -70699.0900 - acc: 0.0000e+00
Epoch 216/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0911 - acc: 0.0000e+00
Epoch 217/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0910 - acc: 0.0000e+00
Epoch 218/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0912 - acc: 0.0000e+00
Epoch 219/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0907 - acc: 0.0000e+00
Epoch 220/1000
3796/3796 [==============================] - 0s 23us/step - loss: -70699.0912 - acc: 0.0000e+00
Epoch 221/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0908 - acc: 0.0000e+00
Epoch 222/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0902 - acc: 0.0000e+00
Epoch 223/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0910 - acc: 0.0000e+00
Epoch 224/1000
3

3796/3796 [==============================] - 0s 24us/step - loss: -70699.0904 - acc: 0.0000e+00
Epoch 289/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0906 - acc: 0.0000e+00
Epoch 290/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0905 - acc: 0.0000e+00
Epoch 291/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0911 - acc: 0.0000e+00
Epoch 292/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0908 - acc: 0.0000e+00
Epoch 293/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0909 - acc: 0.0000e+00
Epoch 294/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0913 - acc: 0.0000e+00
Epoch 295/1000
3796/3796 [==============================] - 0s 23us/step - loss: -70699.0908 - acc: 0.0000e+00
Epoch 296/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0907 - acc: 0.0000e+00
Epoch 297/1000
3

...

3796/3796 [==============================] - 0s 25us/step - loss: -70699.0922 - acc: 0.0000e+00
Epoch 727/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0919 - acc: 0.0000e+00
Epoch 728/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0907 - acc: 0.0000e+00
Epoch 729/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0921 - acc: 0.0000e+00
Epoch 730/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0920 - acc: 0.0000e+00
Epoch 731/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0911 - acc: 0.0000e+00
Epoch 732/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0921 - acc: 0.0000e+00
Epoch 733/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0902 - acc: 0.0000e+00
Epoch 734/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0918 - acc: 0.0000e+00
Epoch 735/1000
3

3796/3796 [==============================] - 0s 25us/step - loss: -70699.0916 - acc: 0.0000e+00
Epoch 800/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0915 - acc: 0.0000e+00
Epoch 801/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0916 - acc: 0.0000e+00
Epoch 802/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0897 - acc: 0.0000e+00
Epoch 803/1000
3796/3796 [==============================] - 0s 26us/step - loss: -70699.0909 - acc: 0.0000e+00
Epoch 804/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0920 - acc: 0.0000e+00
Epoch 805/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0923 - acc: 0.0000e+00
Epoch 806/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0904 - acc: 0.0000e+00
Epoch 807/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0906 - acc: 0.0000e+00
Epoch 808/1000
3

3796/3796 [==============================] - 0s 25us/step - loss: -70699.0919 - acc: 0.0000e+00
Epoch 873/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0917 - acc: 0.0000e+00
Epoch 874/1000
3796/3796 [==============================] - 0s 23us/step - loss: -70699.0903 - acc: 0.0000e+00
Epoch 875/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0920 - acc: 0.0000e+00
Epoch 876/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0905 - acc: 0.0000e+00
Epoch 877/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0901 - acc: 0.0000e+00
Epoch 878/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0914 - acc: 0.0000e+00
Epoch 879/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0908 - acc: 0.0000e+00
Epoch 880/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0909 - acc: 0.0000e+00
Epoch 881/1000
3

3796/3796 [==============================] - 0s 25us/step - loss: -70699.0918 - acc: 0.0000e+00
Epoch 946/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0909 - acc: 0.0000e+00
Epoch 947/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0909 - acc: 0.0000e+00
Epoch 948/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0912 - acc: 0.0000e+00
Epoch 949/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0923 - acc: 0.0000e+00
Epoch 950/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0904 - acc: 0.0000e+00
Epoch 951/1000
3796/3796 [==============================] - 0s 25us/step - loss: -70699.0912 - acc: 0.0000e+00
Epoch 952/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0920 - acc: 0.0000e+00
Epoch 953/1000
3796/3796 [==============================] - 0s 24us/step - loss: -70699.0920 - acc: 0.0000e+00
Epoch 954/1000
3

()

In [ ]:
model3(X_train_new,y_train,X_test_new,y_test)

('Epoch ', 1, '/', 500, '- loss:', 352912437.5)
('Epoch ', 2, '/', 500, '- loss:', 159579653.5)
('Epoch ', 3, '/', 500, '- loss:', 98883713.0)
('Epoch ', 4, '/', 500, '- loss:', 67670538.25)
('Epoch ', 5, '/', 500, '- loss:', 48678936.0)
('Epoch ', 6, '/', 500, '- loss:', 35987758.1875)
('Epoch ', 7, '/', 500, '- loss:', 27294838.1875)
('Epoch ', 8, '/', 500, '- loss:', 20884957.15625)
('Epoch ', 9, '/', 500, '- loss:', 16150374.21875)
('Epoch ', 10, '/', 500, '- loss:', 12539341.375)
('Epoch ', 11, '/', 500, '- loss:', 9805427.828125)
('Epoch ', 12, '/', 500, '- loss:', 7558873.6875)
('Epoch ', 13, '/', 500, '- loss:', 5790254.609375)
('Epoch ', 14, '/', 500, '- loss:', 4283986.9140625)
('Epoch ', 15, '/', 500, '- loss:', 3043832.36328125)
('Epoch ', 16, '/', 500, '- loss:', 2084417.9765625)
('Epoch ', 17, '/', 500, '- loss:', 1441773.876953125)
('Epoch ', 18, '/', 500, '- loss:', 998641.8642578125)
('Epoch ', 19, '/', 500, '- loss:', 719445.9619140625)
('Epoch ', 20, '/', 500, '- los